# DataForSEO | DataForSEO> [DataForSeo](https://dataforseo.com/)は、APIを通じて包括的なSEOおよびデジタルマーケティングのデータソリューションを提供しています。>> > [DataForSeo](https://dataforseo.com/) provides comprehensive SEO and digital marketing data solutions via API.>> `DataForSeo API`は、`Google`、`Bing`、`Yahoo`などの最も人気のある検索エンジンから`SERP`を取得します。また、`Maps`、`News`、`Events`など、異なるタイプの検索エンジンからのSERPも取得できます。>> > The `DataForSeo API` retrieves `SERP` from the most popular search engines like `Google`, `Bing`, `Yahoo`. It also allows to >get SERPs from different search engine types like `Maps`, `News`, `Events`, etc.このノートブックは、[DataForSeo API](https://dataforseo.com/apis)を使用して検索エンジンの結果を取得する方法を示しています。> This notebook demonstrates how to use the [DataForSeo API](https://dataforseo.com/apis) to obtain search engine results.

In [ ]:
from langchain.utilities.dataforseo_api_search import DataForSeoAPIWrapper

## Setting up the API credentials | APIクレデンシャルの設定`DataForSeo`のウェブサイトに登録することで、APIの認証情報を取得できます。> You can obtain your API credentials by registering on the `DataForSeo` website.

In [ ]:
import os

os.environ["DATAFORSEO_LOGIN"] = "your_api_access_username"
os.environ["DATAFORSEO_PASSWORD"] = "your_api_access_password"

wrapper = DataForSeoAPIWrapper()

runメソッドは、answer\_box、knowledge\_graph、featured\_snippet、shopping、organicのいずれかの要素から最初の結果スニペットを返します。> The run method will return the first result snippet from one of the following elements: answer\_box, knowledge\_graph, featured\_snippet, shopping, organic.

In [ ]:
wrapper.run("Weather in Los Angeles")

## The Difference Between `run` and `results` | `run`と`results`の違い`DataForSeoAPIWrapper` クラスが提供する2つのメソッドは `run` と `results` です。> `run` and `results` are two methods provided by the `DataForSeoAPIWrapper` class.`run` メソッドは検索を実行し、回答ボックス、ナレッジグラフ、フィーチャードスニペット、ショッピング、またはオーガニック検索結果から最初の結果スニペットを返します。これらの要素は、最も高い優先度から最も低い優先度に向けて順に並べられます。> The `run` method executes the search and returns the first result snippet from the answer box, knowledge graph, featured snippet, shopping, or organic results. These elements are sorted by priority from highest to lowest.`results` メソッドは、ラッパー内で設定されたパラメータに基づいて構成されたJSONレスポンスを返します。これにより、APIから返すデータをより柔軟に選択することができます。> The `results` method returns a JSON response configured according to the parameters set in the wrapper. This allows for more flexibility in terms of what data you want to return from the API.

## Getting Results as JSON | JSON形式で結果を取得するJSONレスポンスで返される結果のタイプやフィールドをカスタマイズできます。また、返されるトップ結果の数の最大値を設定することも可能です。> You can customize the result types and fields you want to return in the JSON response. You can also set a maximum count for the number of top results to return.

In [ ]:
json_wrapper = DataForSeoAPIWrapper(
    json_result_types=["organic", "knowledge_graph", "answer_box"],
    json_result_fields=["type", "title", "description", "text"],
    top_count=3,
)

In [ ]:
json_wrapper.results("Bill Gates")

## Customizing Location and Language | 位置情報と言語のカスタマイズAPIラッパーに追加のパラメータを渡すことで、検索結果の場所と言語を指定することができます。> You can specify the location and language of your search results by passing additional parameters to the API wrapper.

In [ ]:
customized_wrapper = DataForSeoAPIWrapper(
    top_count=10,
    json_result_types=["organic", "local_pack"],
    json_result_fields=["title", "description", "type"],
    params={"location_name": "Germany", "language_code": "en"},
)
customized_wrapper.results("coffee near me")

## Customizing the Search Engine | 検索エンジンのカスタマイズ使用したい検索エンジンを指定することもできます。> You can also specify the search engine you want to use.

In [ ]:
customized_wrapper = DataForSeoAPIWrapper(
    top_count=10,
    json_result_types=["organic", "local_pack"],
    json_result_fields=["title", "description", "type"],
    params={"location_name": "Germany", "language_code": "en", "se_name": "bing"},
)
customized_wrapper.results("coffee near me")

## Customizing the Search Type | 検索タイプのカスタマイズAPIラッパーを使用すると、実行したい検索のタイプを指定することもできます。たとえば、地図検索を実行することができます。> The API wrapper also allows you to specify the type of search you want to perform. For example, you can perform a maps search.

In [ ]:
maps_search = DataForSeoAPIWrapper(
    top_count=10,
    json_result_fields=["title", "value", "address", "rating", "type"],
    params={
        "location_coordinate": "52.512,13.36,12z",
        "language_code": "en",
        "se_type": "maps",
    },
)
maps_search.results("coffee near me")

## Integration with Langchain Agents | Langchainエージェントとの統合`langchain.agents` モジュールの `Tool` クラスを使用して、`DataForSeoAPIWrapper` をlangchainエージェントと統合することができます。`Tool` クラスは、エージェントが呼び出すことができる関数をカプセル化しています。> You can use the `Tool` class from the `langchain.agents` module to integrate the `DataForSeoAPIWrapper` with a langchain agent. The `Tool` class encapsulates a function that the agent can call.

In [ ]:
from langchain.agents import Tool

search = DataForSeoAPIWrapper(
    top_count=3,
    json_result_types=["organic"],
    json_result_fields=["title", "description", "type"],
)
tool = Tool(
    name="google-search-answer",
    description="My new answer tool",
    func=search.run,
)
json_tool = Tool(
    name="google-search-json",
    description="My new json tool",
    func=search.results,
)